# Run Feast and PSQL

bla bla bla

## Prerequisites

* `Red Hat OpenShift pipelines` operator installed
* Add `admin` role to the ServiceAccount of the current notebook, e.g. something like:
```console
oc adm policy add-cluster-role-to-user -z feast-notebook -n feast admin
```
(this is needed to run `oc` commands from the notebook)

## Install requirements

In [1]:
!cat requirements.txt
!echo '-------------'
!pip install -r requirements.txt

feast==0.37.1
psycopg2>=2.9
-------------


## Create PSQL DB

Create the PSQL DB in the **TARGET_NS** namespace by running the following command to instantiate the application from the template:

In [2]:
# Update it to use a different namespace
%env TARGET_NS=feast

env: TARGET_NS=feast


In [3]:
from IPython.display import Markdown as md
import os

ns = os.environ.get('TARGET_NS')
md(f'''
**Note**: namespace  {ns}  must exist before, otherwise run the following from your CLI logged to the OpenShift console:
```console
oc create ns {ns}
```
''')


**Note**: namespace  feast  must exist before, otherwise run the following from your CLI logged to the OpenShift console:
```console
oc create ns feast
```


In [4]:
!oc process -n openshift postgresql-persistent \
DATABASE_SERVICE_NAME=postgresql POSTGRESQL_USER=feast POSTGRESQL_PASSWORD=feast \
POSTGRESQL_DATABASE=feast VOLUME_CAPACITY=20Gi MEMORY_LIMIT=1Gi | oc apply -f - -n ${TARGET_NS} 

secret/postgresql configured
service/postgresql configured
persistentvolumeclaim/postgresql created
deploymentconfig.apps.openshift.io/postgresql created


Wait until the DB is running:

In [5]:
!oc wait pod -l deploymentconfig=postgresql -n ${TARGET_NS} --for=condition=Ready=true --timeout=5m

error: no matching resources found


## Create sample feast repo

In [16]:
!feast init -m sample_repo


Creating a new Feast repository in /opt/app-root/src/feast-workshop-team-share/feast_showcase_notebook/sample_repo.



Update the default repository to use the local deployment of PSQL DB:

In [17]:
!sed "s/_NAMESPACE_/$TARGET_NS/" templates/feature_store.yaml  > sample_repo/feature_repo/feature_store.yaml
!cat sample_repo/feature_repo/feature_store.yaml

project: feast_postgres
provider: local
registry:
    registry_store_type: PostgreSQLRegistryStore
    path: feast_registry
    host: postgresql.feast.svc.cluster.local
    port: 5432
    database: feast
    db_schema: feast
    user: feast
    password: feast
online_store:
    type: postgres
    host: postgresql.feast.svc.cluster.local
    port: 5432
    database: feast
    db_schema: feast
    user: feast
    password: feast
offline_store:
    type: postgres
    host: postgresql.feast.svc.cluster.local
    port: 5432
    database: feast
    db_schema: feast
    user: feast
    password: feast
entity_key_serialization_version: 2



Update the entity definition from the template

In [18]:
!cp templates/example_repo.py sample_repo/feature_repo

## Validate DB state

Create the function to read the list of tables

In [19]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

psqlHost = 'postgresql.feast.svc.cluster.local'
psqlPort = 5432
psqlUsername = 'feast'
psqlPassword = 'feast'
psqlDb = 'feast'
psqlSchema = 'feast'

In [20]:
# Executes a generic sql_query and return the result as a Pandas DataFrame
def fetchToDF(sql_query):
    engine = create_engine(f'postgresql+psycopg2://{psqlUsername}:{psqlPassword}@{psqlHost}:{str(psqlPort)}/{psqlDb}')

    # SQL command to list tables
    with engine.connect() as conn:
        query = conn.execute(sql_query)
    
    df = pd.DataFrame(query.fetchall())
    return df

In [21]:
# Executes a generic sql_query and return the result as a Pandas DataFrame
def executeSql(sql_command):
    engine = create_engine(f'postgresql+psycopg2://{psqlUsername}:{psqlPassword}@{psqlHost}:{str(psqlPort)}/{psqlDb}')

    # SQL command to list tables
    with engine.connect() as conn:
        conn.execute(sql_command)

In [22]:
# Reads the tables names
def readTables():
    return fetchToDF("SELECT table_name FROM information_schema.tables WHERE table_schema = 'feast';")


Invoke it and verify there are no tables

In [23]:
df = readTables()
assert len(df) == 0, f"Found {len(df)} tables instead of 0"
print('No tables found, as expected')

No tables found, as expected


### Populate offline data from sample parquet

Use `create_driver_hourly_stats_df` to create sample data and push to the data source table `feast_driver_hourly_stats`

In [25]:
from feast.file_utils import replace_str_in_file
from feast.infra.utils.postgres.connection_utils import df_to_postgres_table
from feast.infra.utils.postgres.postgres_config import PostgreSQLConfig
from feast.driver_test_data import create_driver_hourly_stats_df
from datetime import datetime, timedelta

import psycopg2
config_file = "sample_repo/feature_repo/feature_store.yaml"

end_date = datetime.now().replace(microsecond=0, second=0, minute=0)
start_date = end_date - timedelta(days=15)

driver_entities = [1001, 1002, 1003, 1004, 1005]
driver_df = create_driver_hourly_stats_df(driver_entities, start_date, end_date)

tableName = 'feast_driver_hourly_stats'
executeSql(f'DROP TABLE IF EXISTS {tableName}')

df_to_postgres_table(
    config=PostgreSQLConfig(
        host=psqlHost,
        port=psqlPort,
        database=psqlDb,
        db_schema=psqlSchema,
        user=psqlUsername,
        password=psqlPassword,
    ),
    df=driver_df,
    table_name=tableName
)
print(f'Bootstrap completed, added {len(driver_df)} to {tableName}')


Bootstrap completed, added 1807 to feast_driver_hourly_stats


In [26]:
df=fetchToDF('select * from feast_driver_hourly_stats')
df.head()

,event_timestamp,driver_id,conv_rate,acc_rate,avg_daily_trips,created
0,2024-05-15 15:00:00+00:00,1005,0.076286,0.241202,840,2024-05-30 15:26:28.283
1,2024-05-15 16:00:00+00:00,1005,0.632918,0.081792,476,2024-05-30 15:26:28.283
2,2024-05-15 17:00:00+00:00,1005,0.592472,0.328010,446,2024-05-30 15:26:28.283
3,2024-05-15 18:00:00+00:00,1005,0.610005,0.660397,55,2024-05-30 15:26:28.283
4,2024-05-15 19:00:00+00:00,1005,0.130482,0.155755,529,2024-05-30 15:26:28.283


In [27]:
assert len(df) == 1807
print(f'Found {len(df)} items')

Found 1807 items


## Feature store deployment

Now apply the Feast repository and then validate it has the new tables

In [28]:
!feast -c sample_repo/feature_repo apply

/opt/app-root/lib64/python3.9/site-packages/feast/infra/registry/contrib/postgres/postgres_registry_store.py:41: DeprecationWarning: PostgreSQLRegistryStore is deprecated and will be removed in the future releases. Please use SqlRegistry instead.
  warnings.warn(
/opt/app-root/lib64/python3.9/site-packages/feast/feature_store.py:551: RuntimeWarning: On demand feature view is an experimental feature. This API is stable, but the functionality does not scale well for offline retrieval
  warnings.warn(
Deploying infrastructure for driver_hourly_stats_fresh
Deploying infrastructure for driver_hourly_stats


In [29]:
df = readTables()
expected = 4
assert len(df) == expected, f'Found {len(df)} tables, instead of {expected}: {",".join(df["table_name"])}'
print(f'Found {expected} tables, as expected: {",".join(df["table_name"])}')


Found 4 tables, as expected: feast_driver_hourly_stats,feast_registry,feast_postgres_driver_hourly_stats_fresh,feast_postgres_driver_hourly_stats


### Feast state

Verify Feast resources using `feast` CLI

In [30]:
!feast -c sample_repo/feature_repo entities list
!feast -c sample_repo/feature_repo feature-views list
!feast -c sample_repo/feature_repo feature-services list

/opt/app-root/lib64/python3.9/site-packages/feast/infra/registry/contrib/postgres/postgres_registry_store.py:41: DeprecationWarning: PostgreSQLRegistryStore is deprecated and will be removed in the future releases. Please use SqlRegistry instead.
  warnings.warn(
NAME    DESCRIPTION    TYPE
driver                 ValueType.UNKNOWN
/opt/app-root/lib64/python3.9/site-packages/feast/infra/registry/contrib/postgres/postgres_registry_store.py:41: DeprecationWarning: PostgreSQLRegistryStore is deprecated and will be removed in the future releases. Please use SqlRegistry instead.
  warnings.warn(
NAME                         ENTITIES    TYPE
driver_hourly_stats_fresh    {'driver'}  FeatureView
driver_hourly_stats          {'driver'}  FeatureView
transformed_conv_rate        {'driver'}  OnDemandFeatureView
transformed_conv_rate_fresh  {'driver'}  OnDemandFeatureView
/opt/app-root/lib64/python3.9/site-packages/feast/infra/registry/contrib/postgres/postgres_registry_store.py:41: DeprecationWarni

## Install Feature server

Install the [Python feature server](https://docs.feast.dev/reference/feature-servers/python-feature-server)

### Install Helm

In [31]:
!curl https://get.helm.sh/helm-v3.14.3-linux-amd64.tar.gz --output helm.tar.gz
!gunzip -f helm.tar.gz 
!tar xvf helm.tar 
!mv ./linux-amd64/helm .
!./helm version

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 15.3M  100 15.3M    0     0   146M      0 --:--:-- --:--:-- --:--:--  146M
linux-amd64/
linux-amd64/LICENSE
linux-amd64/README.md
linux-amd64/helm
version.BuildInfo{Version:"v3.14.3", GitCommit:"f03cc04caaa8f6d7c3e67cf918929150cf6f3f12", GitTreeState:"clean", GoVersion:"go1.21.7"}


### Install from chart

In [32]:
!./helm repo add feast-charts https://feast-helm-charts.storage.googleapis.com
!./helm repo update

"feast-charts" already exists with the same configuration, skipping
Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "feast-charts" chart repository
Update Complete. ⎈Happy Helming!⎈


In [33]:
import base64
import os

file_path = 'sample_repo/feature_repo/feature_store.yaml'
with open(file_path, 'rb') as file:
    file_content = file.read()

base64_encoded = base64.b64encode(file_content)
os.environ['FEATURE_STORE_YAML_BASE64'] = base64_encoded.decode('utf-8')

In [34]:
!./helm uninstall feast-release

Error: uninstall: Release not loaded: feast-release: release: not found


In [35]:
!echo $FEATURE_STORE_YAML_BASE64

!./helm upgrade --install -n $TARGET_NS feast-release feast-charts/feast-feature-server --set image.tag=0.37.1 --set feature_store_yaml_base64=$FEATURE_STORE_YAML_BASE64

cHJvamVjdDogZmVhc3RfcG9zdGdyZXMKcHJvdmlkZXI6IGxvY2FsCnJlZ2lzdHJ5OgogICAgcmVnaXN0cnlfc3RvcmVfdHlwZTogUG9zdGdyZVNRTFJlZ2lzdHJ5U3RvcmUKICAgIHBhdGg6IGZlYXN0X3JlZ2lzdHJ5CiAgICBob3N0OiBwb3N0Z3Jlc3FsLmZlYXN0LnN2Yy5jbHVzdGVyLmxvY2FsCiAgICBwb3J0OiA1NDMyCiAgICBkYXRhYmFzZTogZmVhc3QKICAgIGRiX3NjaGVtYTogZmVhc3QKICAgIHVzZXI6IGZlYXN0CiAgICBwYXNzd29yZDogZmVhc3QKb25saW5lX3N0b3JlOgogICAgdHlwZTogcG9zdGdyZXMKICAgIGhvc3Q6IHBvc3RncmVzcWwuZmVhc3Quc3ZjLmNsdXN0ZXIubG9jYWwKICAgIHBvcnQ6IDU0MzIKICAgIGRhdGFiYXNlOiBmZWFzdAogICAgZGJfc2NoZW1hOiBmZWFzdAogICAgdXNlcjogZmVhc3QKICAgIHBhc3N3b3JkOiBmZWFzdApvZmZsaW5lX3N0b3JlOgogICAgdHlwZTogcG9zdGdyZXMKICAgIGhvc3Q6IHBvc3RncmVzcWwuZmVhc3Quc3ZjLmNsdXN0ZXIubG9jYWwKICAgIHBvcnQ6IDU0MzIKICAgIGRhdGFiYXNlOiBmZWFzdAogICAgZGJfc2NoZW1hOiBmZWFzdAogICAgdXNlcjogZmVhc3QKICAgIHBhc3N3b3JkOiBmZWFzdAplbnRpdHlfa2V5X3NlcmlhbGl6YXRpb25fdmVyc2lvbjogMgoK
Release "feast-release" does not exist. Installing it now.
NAME: feast-release
LAST DEPLOYED: Thu May 30 15:27:19 2024
NAMESPACE: feast
STATUS: dep

Patch the deployment to silent Feast usage stats (raises a disturbing warning `Certificate did not match expected hostname: usage.feast.dev`)

In [36]:
!oc patch deployment/feast-release-feast-feature-server --type=json --patch '[{"op": "add", "path": "/spec/template/spec/containers/0/env/-", "value": {"name": "FEAST_USAGE", "value": "False"}}]'

deployment.apps/feast-release-feast-feature-server patched


Wait until the server is running

In [37]:
!oc wait pod -l app.kubernetes.io/instance=feast-release -n ${TARGET_NS} --for=condition=Ready=true --timeout=5m

pod/feast-release-feast-feature-server-59fb8b8c59-9mwzm condition met
pod/feast-release-feast-feature-server-5cf4cd587d-7j4p6 condition met


## Integration test

Run use cases and validate using `Feature server` or [Python SDK](https://rtd.feast.dev/en/master/)

### Fetch offline data

#### From server

No API available on the server for offline data

#### From Python SDK

Validate historical features using the data source populated in a [previous step](#Populate-offline-data-from-sample-parquet)

In [38]:
from feast import FeatureStore
from datetime import datetime, timedelta

def fetchHistoricalDataForTest():
    end_date = datetime.now().replace(microsecond=0, second=0, minute=0)
    start_date = end_date - timedelta(days=14)
    test_ts = start_date.replace(hour=6)
    entity_df = pd.DataFrame.from_dict(
        {
            "driver_id": [1001, 1002, 1003],
            "event_timestamp": [
                test_ts,
                test_ts,
                test_ts,
            ],
            "label_driver_reported_satisfaction": [1, 5, 3],
            "val_to_add": [1, 2, 3],
            "val_to_add_2": [10, 20, 30],
        }
    )

    store = FeatureStore(repo_path="sample_repo/feature_repo")

    test_df = store.get_historical_features(
        entity_df=entity_df,
        features=[
            "driver_hourly_stats:conv_rate",
            "driver_hourly_stats:acc_rate",
            "driver_hourly_stats:avg_daily_trips",
            "transformed_conv_rate:conv_rate_plus_val1",
            "transformed_conv_rate:conv_rate_plus_val2",
        ],
    ).to_df()
    return test_df


In [39]:
test_df = fetchHistoricalDataForTest()
test_df.head()

/opt/app-root/lib64/python3.9/site-packages/feast/infra/registry/contrib/postgres/postgres_registry_store.py:41: DeprecationWarning: PostgreSQLRegistryStore is deprecated and will be removed in the future releases. Please use SqlRegistry instead.
  warnings.warn(


,driver_id,event_timestamp,label_driver_reported_satisfaction,val_to_add,val_to_add_2,conv_rate,acc_rate,avg_daily_trips,conv_rate_plus_val1,conv_rate_plus_val2
0,1001,2024-05-16 06:00:00,1,1,10,0.858261,0.437113,618,1.858261,10.858261
1,1002,2024-05-16 06:00:00,5,2,20,0.878192,0.266302,252,2.878192,20.878192
2,1003,2024-05-16 06:00:00,3,3,30,0.615291,0.527527,960,3.615291,30.615291


In [40]:
assert len(test_df) == 3

### Fetch online data

#### From server

In [43]:
import requests

def fetchOnlineFeaturesFromServer():
    payload = {
        "features": [
            "driver_hourly_stats:conv_rate",
            "driver_hourly_stats:acc_rate",
            "driver_hourly_stats:avg_daily_trips"
        ],
        "entities": {
            "driver_id": [1001, 1002, 1003]
        }
    }

    url = "http://feast-release-feast-feature-server.feast.svc.cluster.local/get-online-features"

    return requests.post(url, json=payload)

In [44]:
online = fetchOnlineFeaturesFromServer()

In [45]:
assert online.status_code == 200
try:
    import json

    json_data = json.loads(online.text)
    # print(json.dumps(json_data, indent=2))
    assert "metadata" in json_data
    # Validate metadata
    assert "feature_names" in json_data["metadata"]
    featureNames = json_data["metadata"]["feature_names"]
    assert len(featureNames) == 4
    assert "driver_id" in featureNames
    assert "conv_rate" in featureNames
    assert "acc_rate" in featureNames
    assert "avg_daily_trips" in featureNames
    # Validate data: all NOT_FOUND
    for index, feature in enumerate(featureNames[1:]):
        statuses = json_data["results"][index + 1]["statuses"]
        # print(f'Statuses of {feature}/{index} are {statuses}')
        assert "PRESENT" not in statuses
        assert "NOT_FOUND" in statuses
    print(f'No online data found for all queried features {featureNames}')
except ImportError:
    print(response.text)

No online data found for all queried features ['driver_id', 'avg_daily_trips', 'conv_rate', 'acc_rate']


#### From DB

### Materialize

#### Materialize from server

In [46]:
import requests

end_date = datetime.now().replace(microsecond=0, second=0, minute=0)
start_date = end_date - timedelta(days=15)
start_date = start_date.replace(hour=0)

payload = {
    "start_ts": str(start_date),
    "end_ts": str(datetime.now())
}

url = "http://feast-release-feast-feature-server.feast.svc.cluster.local/materialize"

response = requests.post(url, json=payload)

In [47]:
assert response.status_code == 200

#### Validate from server

In [48]:
online = fetchOnlineFeaturesFromServer()

In [49]:
assert online.status_code == 200
try:
    import json

    json_data = json.loads(online.text)
    # print(json.dumps(json_data, indent=2))
    assert "metadata" in json_data
    # Validate metadata
    assert "feature_names" in json_data["metadata"]
    featureNames = json_data["metadata"]["feature_names"]
    assert len(featureNames) == 4
    assert "driver_id" in featureNames
    assert "conv_rate" in featureNames
    assert "acc_rate" in featureNames
    assert "avg_daily_trips" in featureNames
    # Validate data: all PRESENT
    for index, feature in enumerate(featureNames[1:]):
        statuses = json_data["results"][index + 1]["statuses"]
        # print(f'Statuses of {feature}/{index} are {statuses}')
        assert "PRESENT" in statuses
        assert "NOT_FOUND" not in statuses
    print(f'All online data is present for all queried features {featureNames}')
except ImportError:
    print(response.text)

All online data is present for all queried features ['driver_id', 'avg_daily_trips', 'conv_rate', 'acc_rate']


#### Validate from DB

**TODO** some queries like `select feature_name, event_ts from feast_postgres_driver_hourly_stats;` and validate counters
And the same for the `feast_postgres_driver_hourly_stats_fresh`table

### Push Data

Push sample data on new `driver_id=3001` using Feature server

In [50]:
import requests
import json

push_ts = datetime(2021, 5, 13, 10, 59, 42)
event_dict = {
    "driver_id": [3001],
    "event_timestamp": [str(datetime(2021, 5, 13, 10, 59, 42))],
    "created": [str(push_ts)],
    "conv_rate": [1.0],
    "acc_rate": [1.0],
    "avg_daily_trips": [1000],
}
# "string_feature": "test2",
push_data = {
    "push_source_name":"driver_stats_push_source",
    "df":event_dict,
    "to":"online",
}

# Note: push not implemented ATM for PSQL offline store

url = "http://feast-release-feast-feature-server.feast.svc.cluster.local/push"
response = requests.post(
    url,
    data=json.dumps(push_data))

In [51]:
assert response.status_code == 200

#### Validate from Feature server

In [55]:
import requests

def fetchPushedOnlineFeaturesFromServer():
    payload = {
        "features": [
            "driver_hourly_stats:conv_rate",
            "driver_hourly_stats:acc_rate",
            "driver_hourly_stats:avg_daily_trips"
        ],
        "entities": {
            "driver_id": [3001]
        }
    }

    url = "http://feast-release-feast-feature-server.feast.svc.cluster.local/get-online-features"

    return requests.post(url, json=payload)

In [56]:
pushed = fetchPushedOnlineFeaturesFromServer()

**TODO** Specify to use the `push_ts` for the query


In [62]:
assert pushed.status_code == 200
try:
    import json

    json_data = json.loads(pushed.text)
    # print(json.dumps(json_data, indent=2))
    assert "metadata" in json_data
    # Validate metadata
    assert "feature_names" in json_data["metadata"]
    featureNames = json_data["metadata"]["feature_names"]
    assert len(featureNames) == 4
    assert "driver_id" in featureNames
    assert "conv_rate" in featureNames
    assert "acc_rate" in featureNames
    assert "avg_daily_trips" in featureNames
    # Validate data: all PRESENT
    for index, feature in enumerate(featureNames[1:]):
        statuses = json_data["results"][index + 1]["statuses"]
        # print(f'Statuses of {feature}/{index} are {statuses}')
        assert "PRESENT" in statuses
        assert "NOT_FOUND" not in statuses
    print(f'No online data found for all queried features {featureNames}')
except ImportError:
    print(response.text)

AssertionError: 

## Tear down

Tear down deployed feature store infrastructure

In [64]:
!feast --log-level DEBUG -c sample_repo/feature_repo teardown

/opt/app-root/lib64/python3.9/site-packages/feast/infra/registry/contrib/postgres/postgres_registry_store.py:41: DeprecationWarning: PostgreSQLRegistryStore is deprecated and will be removed in the future releases. Please use SqlRegistry instead.
  warnings.warn(
05/30/2024 03:32:00 PM feast.infra.registry.registry INFO: Registry cache expired, so refreshing
05/30/2024 03:32:00 PM feast.infra.registry.registry INFO: Registry cache expired, so refreshing
05/30/2024 03:32:00 PM feast.infra.registry.registry INFO: Registry cache expired, so refreshing
^C


Remove local project

In [65]:
!rm -rf sample_repo

Uninstall `Feature server`

In [66]:
!./helm uninstall -n $TARGET_NS feast-release

release "feast-release" uninstalled


Uninstall PSQL DB

In [67]:
!oc process -n openshift postgresql-ephemeral \
DATABASE_SERVICE_NAME=postgresql POSTGRESQL_USER=feast POSTGRESQL_PASSWORD=feast \
POSTGRESQL_DATABASE=feast | oc delete -f - -n ${TARGET_NS} 

secret "postgresql" deleted
service "postgresql" deleted
deploymentconfig.apps.openshift.io "postgresql" deleted
